In [1]:
from pathlib import Path
import json
import os
import csv

In [2]:
# Define paths to the main repositories
images_base_path = Path('../../training/DATA/UAV-benchmark-M')
annotations_base_path = Path('../../training/Data/UAV-benchmark-MOTD_v1.0/GT')

# Initialize the dictionary
annotations_to_images = {}

# Iterate through each subdirectory in the images base path
for image_dir in images_base_path.iterdir():
    if image_dir.is_dir():  # Ensure it's a directory
        # Construct the corresponding CSV file path
        csv_file = annotations_base_path / f"{image_dir.name}_gt_whole.txt"
        
        # Collect all image paths in the current directory
        image_paths = list(image_dir.glob('*.jpg'))
        
        # Add to the dictionary
        annotations_to_images[str(csv_file)] = [str(img_path) for img_path in image_paths]

# Print the resulting dictionary
for csv_file, image_paths in annotations_to_images.items():
    print(f"CSV File: {csv_file}")
    for img_path in image_paths:
        print(f"  Image: {img_path}")


KeyboardInterrupt: 

In [ ]:
# Function to parse a bounding box line from a TXT file
def parse_bbox_line(line):
    parts = line.strip().split(',')
    frame_index = int(parts[0])
    target_id=int(parts[1])
    bbox_left = float(parts[2])
    bbox_top = float(parts[3])
    bbox_width = float(parts[4])
    bbox_height = float(parts[5])
    x1, y1 = bbox_left, bbox_top
    x2, y2 = bbox_left + bbox_width, bbox_top + bbox_height
    return frame_index, (x1, y1, x2, y2),int(parts[8]),target_id



In [ ]:

frame_to_bboxes=[]
data_list=[]

# Process the dictionary to create pairs of image paths and bounding boxes
for txt_file, image_paths in annotations_to_images.items():
    if Path(txt_file).exists():
        # Parse the TXT file to get bounding boxes
        with Path(txt_file).open() as f:
            bboxes = [parse_bbox_line(line) for line in f]
        
        # Create a mapping from frame index to bounding boxes
        frame_to_bboxes = {index: [frame_index,bbox,class_id,target_id]for index,(frame_index, bbox ,class_id,target_id) in enumerate(bboxes)}
        
        # Match images with bounding boxes based on frame index
        for i,img_path in enumerate(image_paths):
            frame_index = int((Path(img_path).stem)[3:])
            keys_with_value_x = [key for key, value in frame_to_bboxes.items() if value[0] == frame_index]
            if len(keys_with_value_x)>0:
                for key in keys_with_value_x:
                    img_pair={
                        'image_path':img_path,
                        'xmin':frame_to_bboxes[key][1][0],
                        'ymin':frame_to_bboxes[key][1][1],
                        'xmax':frame_to_bboxes[key][1][2],
                        'ymax':frame_to_bboxes[key][1][3]
                            }
                    data_list.append(img_pair)
                # filename = img_path


In [ ]:
# Write to CSV
csv_file = "bounding_boxes.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["image_path", "xmin", "ymin", "xmax", "ymax"])
    writer.writeheader()
    for row in data_list:
        writer.writerow(row)